In [2]:
from fastcore.utils import *
from fastcore.dispatch import *

In [3]:
typedispatch??

Signature:   typedispatch(f)
Type:        DispatchReg
String form: <fastcore.dispatch.DispatchReg object at 0x7fbb36a08290>
File:        ~/dev/env37/lib/python3.7/site-packages/fastcore/dispatch.py
Source:     
class DispatchReg:
    "A global registry for `TypeDispatch` objects keyed by function name"
    def __init__(self): self.d = defaultdict(TypeDispatch)
    def __call__(self, f):
        nm = f'{f.__qualname__}'
        self.d[nm].add(f)
        return self.d[nm]


In [4]:
@typedispatch
def hello(x:int,y:str):
    print('intx',y)
    
@typedispatch
def hello(x:float,y:str):
    print('floatx',y)
    
print(hello)

(float,str) -> hello
(int,str) -> hello


In [5]:
hello(1,'y')

intx y


In [6]:
hello(1.,'y')

floatx y


In [7]:
@typedispatch
def hello(x:(float,int),y:str):
    print('float,int x',y)
@typedispatch
def hello(x:(int,float),y:str):
    print('int, float x',y)

print(hello)

(float,str) -> hello
(int,str) -> hello


In [8]:
hello(1,'y')#bug???

int, float x y


In [9]:
hello(1.,'y')#bug???

int, float x y


In [10]:
hello((1.,1),'y') # it does not reconize within the tuple

(1.0, 1)

In [11]:
hello((1,1.),'y')

(1, 1.0)

In [12]:
hello(tuple(''),'y')

()

In [13]:
@typedispatch
def hello(x,y):
    print('unknown',y)
    

In [14]:
hello(1,'y')

int, float x y


In [15]:
hello(1.,'y')

int, float x y


In [16]:
print(hello)

(int,str) -> hello
(float,str) -> hello
(object,object) -> hello


In [23]:
from fastcore.dispatch import _p2_anno, _TypeDict

In [40]:
# @typedispatch
def f(x:(int,float),y:str,*args):
    print([type(o) for o in x] , 'wo dispatch')
a0,a1 = _p2_anno(f)
funcs = _TypeDict()
print(funcs)
t = None
if t is None:
    t = _TypeDict()
    print(a0)
    funcs.add(a0, t)
t.add(a1, f)
print(funcs)

{}
(<class 'int'>, <class 'float'>)
{<class 'float'>: {<class 'str'>: <function f at 0x7fbb36980440>}, <class 'int'>: {<class 'str'>: <function f at 0x7fbb36980440>}}


In [29]:
print(funcs)

{<class 'int'>: {<class 'str'>: <function f at 0x7fbb36980830>}}


In [34]:
@typedispatch
def tuple_inside(x:(int,float),y:str):
    if type(x) is tuple:
        print([type(o) for o in x] , 'w dispatch')
    print(type(x), 'w distpatch')
print(_p2_anno(tuple_inside))

[<class 'object'>, <class 'object'>]


In [35]:
tuple_inside((1,1.),'')

(1, 1.0)

In [36]:
tuple_inside(1,'')

<class 'int'> w distpatch


In [37]:
tuple_inside(1.,'')

<class 'float'> w distpatch


In [38]:
# So it just adds both types in the tuple???
# is this intented?

In [41]:
# yes this is (semi) intented!
# _TypeDict.add :
def add(self, t, f):
    "Add type `t` and function `f`"
    if not isinstance(t,tuple): t=tuple(L(t))
    for t_ in t: self.d[t_] = f
    self._reset()

In [54]:
from fastai2.basics import *

/home/tako/dev/env37/lib/python3.7/site-packages/pandas/compat/__init__.py:117: UserWarning: Could not import the lzma module. Your installed Python is incomplete. Attempting to use lzma compression will result in a RuntimeError.
  warnings.warn(msg)


In [55]:
o = TensorBase([1], meta='1')
o, o._meta

(TensorBase([1]), {'meta': '1'})

In [56]:
class MetaTuple(Tuple):
    def __new__(cls, x, *rest, **kwargs):
        r = Tuple.__new__(cls,x, *rest)
        r._meta = {i:a._meta for i,a in enumerate(L(r)) if hasattr(a,'_meta')}
        r._types = [type(a) for a in L(r)]
        return r

In [57]:
o = MetaTuple(1)
o, o._meta, o._types

((1), {}, [int])

In [58]:
a = TensorBase(tensor(1), label='x')
b = TensorBase(tensor(1), label='y')
print(a._meta)
o = MetaTuple((a,b))
o, getattr(o,'_meta',None), o._types

{'label': 'x'}


((TensorBase(1), TensorBase(1)),
 {0: {'label': 'x'}, 1: {'label': 'y'}},
 [fastai2.torch_core.TensorBase, fastai2.torch_core.TensorBase])